In [1]:
# coding=utf-8

In [2]:
# imports libraries of python
from PIL import Image
import pytesseract
from pytesseract import image_to_string
import requests
from io import BytesIO
import sys
import re
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import time
from getpass import getpass

# Defining a function to check the validity of Xpath in the browser
def hasXpath(xpath):
    try:
        if wd.find_element(By.XPATH, xpath):
            return True
    except:
        return False


# Defining a function to check the validity of Xpath in the Post
def posthasXpath(xpath,post):
    try:
        if post.find_element(By.XPATH, xpath):
            return True
    except:
        return False

    
# Defining a function to check if facebook is logged in
def is_logged_in(driver):
    try:
        profile_svg_element = driver.find_element(By.XPATH, './/div[@aria-label="Account Controls and Settings"]')
        return True
    except NoSuchElementException:
        return False

    
# Defining a function to login
def login():
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    global wd
    wd = webdriver.Chrome(options=chrome_options)


    # Taking to the log-in page
    wd.get('https://www.facebook.com')

    # username and password
    usr = input('Enter username:\n')  # Username
    pas = getpass('Enter Password:\n')  # password

    username = wd.find_element(By.ID, 'email')
    username.send_keys(usr)
    sleep(0.5)

    # Locate password form by ID
    passw = wd.find_element(By.ID, 'pass')
    passw.send_keys(pas)
    sleep(0.5)

    # locate submit button by_xpath
    sign_in_button = wd.find_element(By.XPATH, '//*[@type="submit"]')
    sign_in_button.click()
    sleep(3)
    
    if is_logged_in(wd):
        print("logged in")
    else:
        print('log_in Error')

        
# Defining a function for initializing the search for a given keyword
def initialize_search(keyword):
    print("----------------------------xxxx----------------------------")
    print(f'Searching for {keyword}')

    # Open the file in write mode with the keyword provided
    global f 
    f = open(keyword + "_facebook.txt", "w", encoding="utf-8")
    
    head = f"'post_no', 'post_type', 'is_text_only' ,'user_name', 'post_time','likes','num_coments','num_shares','post_content','Cmt_content','post_url'\n"
    f.write(head)
    
    # Entering search page for facebook with the keyword
    wd.get('https://www.facebook.com/search/posts?q=' + keyword)

    

def scroll_to_bottom(wd, num_scrolls=3):
    j = 0
    while j < num_scrolls:
        # Scroll down to bottom so as to load more content
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(2)
        j = j + 1
    sleep(2)
    
    
def scroll_into_view(element):
    wd.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    
    
def click_see_more(element):
    while hasXpath("//*[contains(text(), 'See more')]"):
            seemore_buttons = element.find_elements(By.XPATH, "//*[contains(text(), 'See more')]")
            print(len(seemore_buttons))
            for seemore_button in reversed(seemore_buttons):
                scroll_into_view(seemore_button)
                sleep(1)
                seemore_button.click()
    

def extract_user_name(name,post_type):
    user_name = None
    if post_type != "reel":
        user_name = name.text
        if not user_name:
            Element_HTML = name.get_attribute("outerHTML")
            user_name = Element_HTML.strip("</span>")
    
    elif post_type == "reel":
        Element_HTML = name.get_attribute("outerHTML")
        match = re.search(r'target="_blank">(.*?)</span>', Element_HTML)
        if match:
            user_name= match.group(1)
        elif not user_name:
            match = re.search(r'target="_blank">(.*?)</a>', Element_HTML)
            if match:
                user_name= match.group(1)
    return user_name


def extract_post_content(post):
    post_content = ""
    post_texts = post.find_elements(By.XPATH, post_content_xpath)
    for post_text in post_texts:
        content = None
        content = post_text.text
        if not content:
            Element_HTML = post_text.get_attribute("outerHTML")
            match = re.search(r'text-align: start;">(.*?)<', Element_HTML)
            if match:
                content = match.group(1)
            else:
                match = re.search(r'text-align: start;">(.*?)<', Element_HTML)
                if match:
                    content = match.group(1)
                else:
                    match = re.search(r'tabindex="0">(.*?)<', Element_HTML)
                    if match:
                        content = match.group(1)
                    else:
                        content= Element_HTML
            post_content = post_content + content + " "
    return post_content
    

def extract_reels_info(post):
    #post_time for reels
    reel_date = post.find_element(By.XPATH, reel_date_xpath)
    Element_HTML = reel_date.get_attribute("outerHTML")
    post_time = re.search(r'x1qrby5j">(.*?)</span>', Element_HTML).group(1)

    #link for reels
    if (posthasXpath(reels_link_xpath, post)):
        allp = post.find_element(By.XPATH, reels_link_xpath)
        post_url = allp.get_attribute("href")

    other_content = post.find_elements(By.XPATH, './/span[@class="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft x1j85h84"]')
    content_text = ""
    for content in other_content:
        content_line = content.text
        if not content_line:
            content_line = content.get_attribute("outerHTML")
        content_text = content_text + content_line + "\n"
        
    return post_time, post_url, content_text


def extract_interaction_data(post):
    likes, num_comments, num_shares = "","",""
    #No of likes for post
    if posthasXpath(likes_num_xpath, post):
        No_likes = post.find_element(By.XPATH, likes_num_xpath)
        likes = No_likes.text
        if not likes:
            Element_HTML = No_likes.get_attribute("outerHTML")
            match = re.search(r'<span class="x1e558r4">(.*?)</span>', Element_HTML)
            if match:
                likes = match.group(1)

    #no. of comments and shares                
    if posthasXpath(comment_num_xpath, post) and post_type != "reel":
        No_comments = post.find_elements(By.XPATH, comment_num_xpath)
        if len(No_comments)>1:
            num_comments= No_comments[1].text 
            if not num_comments:
                Element_HTML = No_comments[1].get_attribute("outerHTML")
                match = re.search(r'dir="auto">(.*?)</span>', Element_HTML)
                if match:
                    num_comments = match.group(1)

        if len(No_comments)>2:
            num_shares= No_comments[2].text 
            if not num_shares:
                Element_HTML = No_comments[2].get_attribute("outerHTML")
                match = re.search(r'dir="auto">(.*?)</span>', Element_HTML)
                if match:
                    num_shares = match.group(1)
        
    return likes, num_comments, num_shares



In [3]:
####        
# main 
try:
    if is_logged_in(wd):
        print("Already logged in")
        
except:
    login()
    
keyword = [ "stock market fraud" , "stock market predictions", "stock market Equity", "stock market Broker","stock market Arbitrage", "stock market Trading", "Prediction","Dividends","stockmarket", "shear-market", "nifty", "banknifty", "investment", "intraday", "options trading"]

Enter username:
arushinde789@gmail.com
Enter Password:
········
logged in


In [ ]:
fb_feed_xpath = '//div[@class="x193iq5w x1xwk8fm"]'
fb_artical_xpath = '//div[@class="x1a2a7pz"]'
general_u_name_xpath = './/span[@class="xt0psk2"]/a/strong/span'
group_post_u_name_xpath = './/span[@class="xt0psk2"]/a/span'
text_post_xpath = './/div[@class="xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs"]'
reels_u_name_xpath = './/span[@class="x65f84u"]//a'
reels_link_xpath = './/div[@class="x1n2onr6"]/div[1]/div/a[@role = "link"]'
reel_date_xpath = './/span[@class="x4k7w5x x1h91t0o x1h9r5lt x1jfb8zj xv2umb2 x1beo9mf xaigb6o x12ejxvf x3igimt xarpa2k xedcshv x1lytzrv x1t2pt76 x7ja8zs x1qrby5j"]'
post_content_xpath =  './/div[@class="xu06os2 x1ok221b"]/span/div/div'
likes_num_xpath = './/span[@class="x1e558r4"]'
comment_num_xpath = './/div[@class="x1n2onr6"]/div/div[2]/div'
link_url_xpath = './/a[@class = "x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm" and @role = "link" and @tabindex="0"]'
                

for i in range(0, len(keyword)):
    initialize_search(keyword[i])
    NUM_SCROLLS =10                    # change with no. of scrolls
    scroll_to_bottom(wd,NUM_SCROLLS)   
    sleep(2)
    try:
        click_see_more(wd)
    except:
        print("failed to click see more")
            
# --------------------------------xxxxxx----------------------------------------

    if hasXpath(fb_feed_xpath):   #xpath for facebbok feed:

        
        posts = wd.find_elements(By.XPATH, fb_artical_xpath)   #xpath for facebbok artical:
        print(f'has {len(posts)}  post')
        sleep(5)
        for post_no, post in enumerate(posts):
            print('\n\n----------xx----------')
            print(f'\nlooking inside post no: {post_no} ')
            
            post_type, is_text_only = "Unknown" , 0
            user_name,post_time,likes,num_comments,num_shares,post_content,Cmt_content,post_url = "","","","","","","",""

            # Taking the user name from the post with the Xpath provided
            if posthasXpath(general_u_name_xpath, post):
                post_type = "general"
                name = post.find_element(By.XPATH, general_u_name_xpath)
                user_name = extract_user_name(name,post_type)
                likes, num_comments, num_shares = extract_interaction_data(post)
                    
            
            if posthasXpath(group_post_u_name_xpath, post):
                post_type = "group_post"
                name = post.find_element(By.XPATH, group_post_u_name_xpath)
                user_name = extract_user_name(name,post_type)
                likes, num_comments, num_shares = extract_interaction_data(post)
            
            if posthasXpath(reels_u_name_xpath, post):
                post_type = "reel"
                name = post.find_element(By.XPATH, reels_u_name_xpath)
                user_name = extract_user_name(name,post_type)
                post_time, post_url, post_content = extract_reels_info(post)            
            
            
            # post_content
            if posthasXpath(post_content_xpath, post):
                post_content = extract_post_content(post)
                print(post_content)
            
            if posthasXpath(text_post_xpath, post):
                is_text_only = 1
                post_text = post.find_element(By.XPATH, text_post_xpath)
                post_content = post_text.text
                if not post_content:
                    Element_HTML = post_text.get_attribute("outerHTML")
                    match = re.search(r'">(.*?)</div>', Element_HTML)
                    if match:
                        post_content= match.group(1)
                        
            
                

            # links and post_times for other post
            if posthasXpath('.//a[@role = "link" and @tabindex="0"]', post) and post_type != 'reel':
                all_a_s = post.find_elements(By.TAG_NAME, 'a')    # Find all a elements 
                for a_no,a_s in enumerate(all_a_s):
                    post_url = a_s.get_attribute("href")
                    print("a_no:\t ", a_no,"\nurl: ", post_url)
                post_url = all_a_s[-3].get_attribute("href")
                total_a = len(all_a_s)
                if total_a>4:
                    a_s= all_a_s[4].get_attribute("outerHTML")
                    svg_id = re.search(r'<use xlink:href="#SvgT(\d)" xmlns:xlink="http://www.w3.org/1999/xlink"></use>', a_s)
                    if svg_id:
                        svg = int(svg_id.group(1))
                        date = wd.find_elements(By.ID, f"SvgT{svg}")
                        Element_HTML = date[0].get_attribute("outerHTML")
                        post_time = re.search(r'visible;">(.*?)</text>', Element_HTML)
                        if post_time:
                            post_time= post_time.group(1)
                            print("Post Time : ", post_time)
                    if not svg_id:
                        a_s= all_a_s[3].get_attribute("outerHTML")
                        svg_id = re.search(r'<use xlink:href="#SvgT(\d)" xmlns:xlink="http://www.w3.org/1999/xlink"></use>', a_s)
                        if svg_id:
                            svg = int(svg_id.group(1))
                            date = wd.find_elements(By.ID, f"SvgT{svg}")
                            Element_HTML = date[0].get_attribute("outerHTML")
                            post_time = re.search(r'# visible;">(.*?)</text>', Element_HTML)
                            if post_time:
                                post_time= post_time.group(1)
                                print("Post Time : ", post_time)


        
            print('opening url')
            if post_url:
                try:# Open a new tab
                    wd.execute_script("window.open();")
                    # Switch to the newly opened tab
                    new_tab = wd.window_handles[-1]
                    wd.switch_to.window(new_tab)
                    wd.get(post_url)
                    print("new link loaded")
                    sleep(5)
                    if hasXpath( '//div[@class= "x78zum5 xdt5ytf x6ikm8r x1odjw0f x1iyjqo2 x1pi30zi x1swvt13"]'):
                        print("has_comments session")
                        commets_session = wd.find_element(By.XPATH, '//div[@class= "x78zum5 xdt5ytf x6ikm8r x1odjw0f x1iyjqo2 x1pi30zi x1swvt13"]')
                    try:
                        button_element = wd.find_element(By.XPATH, './/div[@class = "x6s0dn4 x78zum5 xdj266r x11i5rnm xat24cr x1mh8g0r xe0p6wg"]/div[@role= "button"]')
                        button_element.click()
                        sleep(0.5)
                    except:
                        print("it had error 1") 
                    else:
                        try:
                            button_element = wd.find_element(By.XPATH, './/div[@role = "menu" and @class = "x1n2onr6 xcxhlts x1fayt1i"]//div[@class = "x78zum5 xdt5ytf x1iyjqo2 x1n2onr6"]/div/div[3]')
                            button_element.click()   
                            print("sucses1")
                            sleep(3)
                        except:
                            print("it had error 2")
                        else:
                            try:
                                while posthasXpath('.//div[@class="x78zum5 x1iyjqo2 x21xpn4 x1n2onr6"]/div[@role= "button"]',commets_session):
                                    View_reply_buttons = commets_session.find_elements(By.XPATH, './/div[@class="x78zum5 x1iyjqo2 x21xpn4 x1n2onr6"]/div[@role= "button"]')
                                    print("number of buttons detected: ",len(View_reply_buttons))
                                    for View_reply_button in View_reply_buttons:
                                        scroll_into_view(View_reply_button)
                                        View_reply_button.click()   
                                        sleep(0.5)
                            except:
                                print("it had error 3")
                                
#                     if hasXpath( '//div[@class= "x78zum5 xdt5ytf x6ikm8r x1odjw0f x1iyjqo2 x1pi30zi x1swvt13"]'):
#                         print("has_comments session")
#                         commets_session = wd.find_element(By.XPATH, '//div[@class= "x78zum5 xdt5ytf x6ikm8r x1odjw0f x1iyjqo2 x1pi30zi x1swvt13"]')
#                         try:
#                             click_see_more(commets_session)
#                         except:
#                             pass


                    if hasXpath('//div[@class="x1n2onr6 x1swvt13 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz" and @role="article"]'):
                        comments = wd.find_elements(By.XPATH, '//div[@class="x1n2onr6 x1swvt13 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz" and @role="article"]')
                        print(f"has {len(comments)} comments")
                        Cmt_content = ""
                        for x,y in enumerate( comments):
                            try:
                                commenter_name = y.find_element(By.XPATH, './/span[@class = "x3nfvp2"]/span' )
                                cmt_user_name= commenter_name.text
                            except:
                                cmt_user_name = "is_top_fan"
                            if posthasXpath('.//div[@class= "x1lliihq xjkvuk6 x1iorvi4"]/span/div/div',y):
                                comment_lines = y.find_elements(By.XPATH, './/div[@class= "x1lliihq xjkvuk6 x1iorvi4"]/span/div/div' )
                                com_lines = ""
                                for line in comment_lines:
                                    line_text = line.text
                                    com_lines = com_lines+line_text
                            Cmt_content= Cmt_content+f" comment_{x+1}, {cmt_user_name} : {com_lines} | "
#                         print(Cmt_content)
                
                except KeyboardInterrupt:
                    print("Keyboard Interrupt")
                        
                finally:
                    # Close the new tab
                    wd.close()
                    # Switch back to the original tab
                    original_tab = wd.window_handles[0]
                    wd.switch_to.window(original_tab)
                    


                
            to_write = f"'post_{post_no}:', '{post_type}','{is_text_only}' ,'{user_name}', '{post_time}','{likes}','{num_comments}','{num_shares}' ,'{post_content}','{Cmt_content}','{post_url}'\n"
            print(f"'post_{post_no}:',\nPost_type:\t\t'{post_type}',\nis_text_only:\t\t{is_text_only},\nUserName:\t\t'{user_name}',\nPost_time:\t\t'{post_time}',\nNo_likes:\t\t'{likes}',\nnumber_comments:\t'{num_comments}',\nnum_shares:\t\t'{num_shares}'  ,\nPost_content:\t'{post_content}',\nComments:\t'{Cmt_content}',\n{post_url}\n\n")
            cleaned_text = to_write.replace("\n", "  ").replace("\t", "  ")
            f.write(f'{cleaned_text} \n')
            
            

----------------------------xxxx----------------------------
Searching for stock market fraud
1
has 5  post


----------xx----------

looking inside post no: 0 
<div class="x9f619 x1n2onr6 x1ja2u2z x78zum5 xdt5ytf x193iq5w xeuugli x1r8uery x1iyjqo2 xs83m0k"><h2 class="x1heor9g x1qlqyl8 x1pd3egz x1a2a7pz x193iq5w xeuugli"><span class="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lkfr7t x1lbecb7 x1s688f xzsf02u"><span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6 x1j85h84" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">New</span></span></h2></div>  
a_no:	  0 
url:  https://www.facebook.com/notifications/
a_no:	  1 
url:  https://www.facebook.com/watch/?v=300234789273497&aggr_v_ids%5B0%5D=300234789273497&notif_id=1706072648739075&notif_t=watch_follower_video&ref=notif
a_no:	  2 
url:  https://www.facebook.com/reel/3634435133544492?s=notification&notif_id=1706024279704022&notif_t=fb_shorts_follower_posted&ref=notif
a_no:	  3 
url:  https://www.facebook.com/

new link loaded
has_comments session
it had error 1
'post_3:',
Post_type:		'general',
is_text_only:		0,
UserName:		'Mycapital89',
Post_time:		'None',
No_likes:		'4',
number_comments:	'',
num_shares:		''  ,
Post_content:	'',
Comments:	'',
https://www.facebook.com/100086701407226/videos/1538333149924607/?__cft__[0]=AZVUwwc0coAesYnUKc8HF7PPiBrEM5Qqq-YheC_14vvEivzOB6s0k73AvsFCOJHM1KdHHbWlj2EtyDnGU4ZhX_a4rkCsmiyv6VYRyMmYGns_HTgB8sUTU3j1JFoqiqe3q_ZKwzPnpwVVlnrhtIiE4S9GrohV1QElG5-Y8PfWQ7DGFZP4YSm9QaG3MWIGGyzliXw&__tn__=%2B%3FFH-R




----------xx----------

looking inside post no: 4 

a_no:	  0 
url:  https://www.facebook.com/sharenewstoday?__cft__[0]=AZV20ViX5qcXt1anKLSNjKn62XqPhOcXMEVDbtDR6yxoVIGxmNhAygebu92uUEsnsrNYc9wiS5Mq1d8OSKeNSMPn2RWssLyXqkUY4kPHjC6QAoJs2MWHGrH_OWODb055dlD8CAygBxuywcKigNhKefreG7gy3BrmIuNLBKENwFvOg5nSaIPLP_vcNrv2v6hQHv8&__tn__=%3C%2CP-R
a_no:	  1 
url:  https://www.facebook.com/sharenewstoday?__cft__[0]=AZV20ViX5qcXt1anKLSNjKn62XqPhOcXMEVDbtDR6yxoVIGxmNhAygebu92uUEsnsr

new link loaded
it had error 1
'post_0:',
Post_type:		'Unknown',
is_text_only:		0,
UserName:		'',
Post_time:		'',
No_likes:		'',
number_comments:	'',
num_shares:		''  ,
Post_content:	'<div class="x9f619 x1n2onr6 x1ja2u2z x78zum5 xdt5ytf x193iq5w xeuugli x1r8uery x1iyjqo2 xs83m0k"><h2 class="x1heor9g x1qlqyl8 x1pd3egz x1a2a7pz x193iq5w xeuugli"><span class="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lkfr7t x1lbecb7 x1s688f xzsf02u"><span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6 x1j85h84" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">New</span></span></h2></div>  ',
Comments:	'',
https://www.facebook.com/login_alerts/start/?fbid=2007373666314250&s=j&notif_id=1705138721624221&notif_t=login_alerts_new_device&ref=notif




----------xx----------

looking inside post no: 1 
opening url
'post_1:',
Post_type:		'Unknown',
is_text_only:		0,
UserName:		'',
Post_time:		'',
No_likes:		'',
number_comments:	'',
num_shares:		''  ,
Post_content:	'',
Comments:	'',

opening url
new link loaded
has_comments session
sucses1
number of buttons detected:  1
has 4 comments
'post_3:',
Post_type:		'general',
is_text_only:		0,
UserName:		'Stock Market Daddy',
Post_time:		'None',
No_likes:		'28',
number_comments:	'4 comments',
num_shares:		'1 share'  ,
Post_content:	'  ',
Comments:	' comment_1, Ratnakar Dorkar : Super Dada |  comment_2, Neeraj Kumar : Super |  comment_3, Krishna Jadhav : Cool |  comment_4, Pavan Malji : Cool | ',
https://www.facebook.com/Bankniftycalls.IN/videos/1514282235808340/?__cft__[0]=AZWkQTLPeQHPMg_L13RN8qmNxMPoGFlEJVsjN6N3BaAqKle4UeVBp4cHQ8qsOYgg_1sfm6x41ekqJBt4g-rXr-nrXpEfDhZoCKK3noQcwanNRJWnLaKgp_Sc1TUrVGBiuhFoPeO9McC59LUkfk9kzzIfYbYocmZv72cWGyIB2lric7RHgEwPJhAzyv3UEeUBRLQ&__tn__=%2B%3FFH-R




----------xx----------

looking inside post no: 4 

a_no:	  0 
url:  https://www.facebook.com/forexstockpredictions?__cft__[0]=AZVSGf-y1PVK_Dsp1uE7Wuzaa0VcXI8eeM6ePERRrxEUtpxrnBfNqcwtPnbGSZDwu-xH3-QsmnX2j73m9ILPVMpyxwy4K22qtxlHIInAcs5Q3D8bf


a_no:	  0 
url:  https://www.facebook.com/groups/506865050937818/?hoisted_section_header_type=recently_seen&multi_permalinks=939688204322165&__cft__[0]=AZUZN2zrtNJLhHBZrEdCMQza6ASG2QU0bxtXaGxUyRWrL5JrtZUb6-o2y-qQ6ZLlR7haFqdjNicGqQ8JsCw4BoASZ3DEPKstu6y1e0s2faKkU67ESnjb3lFhkVnRDovt8dryMANZmLVC2cgviOTI0KJoxKKtDhGYf3WjqN4PNqLOe_BpTZesXX0EpHvEuhYups0&__tn__=%3C%3C%2CP-R
a_no:	  1 
url:  https://www.facebook.com/groups/506865050937818/user/100035862977307/?__cft__[0]=AZUZN2zrtNJLhHBZrEdCMQza6ASG2QU0bxtXaGxUyRWrL5JrtZUb6-o2y-qQ6ZLlR7haFqdjNicGqQ8JsCw4BoASZ3DEPKstu6y1e0s2faKkU67ESnjb3lFhkVnRDovt8dryMANZmLVC2cgviOTI0KJoxKKtDhGYf3WjqN4PNqLOe_BpTZesXX0EpHvEuhYups0&__tn__=%3C%2CP-R
a_no:	  2 
url:  https://www.facebook.com/groups/506865050937818/?hoisted_section_header_type=recently_seen&multi_permalinks=939688204322165&__cft__[0]=AZUZN2zrtNJLhHBZrEdCMQza6ASG2QU0bxtXaGxUyRWrL5JrtZUb6-o2y-qQ6ZLlR7haFqdjNicGqQ8JsCw4BoASZ3DEPKstu6y1e0s2faKkU67ESnjb3lFhkVnRDovt8dryMANZmLVC2cgviOTI0KJoxKKtDhGYf3WjqN4P

...

a_no:	  28 
url:  https://www.facebook.com/hashtag/niftyoptions?__eep__=6&__cft__[0]=AZUGbvu0df-ygzeWzg-2AOcDtnOaWXQKTDMHhSfbI5ALwRCh2AAEQWQQCc6NaQXoI_P8rzO6Q6Jd9m7p_MckumNKw5MCyzm9WDHX_u-d_0T9l7VzsF0Bn885vFM95sSqkjsxdwqe0LrmoCFhAPNG9Y0ScdjhjOq4ytpCrpQPS4gnTwJ50JnSNH1kL3kuqp0kSWo&__tn__=*NK-R
a_no:	  29 
url:  https://www.facebook.com/hashtag/sharmarkettips?__eep__=6&__cft__[0]=AZUGbvu0df-ygzeWzg-2AOcDtnOaWXQKTDMHhSfbI5ALwRCh2AAEQWQQCc6NaQXoI_P8rzO6Q6Jd9m7p_MckumNKw5MCyzm9WDHX_u-d_0T9l7VzsF0Bn885vFM95sSqkjsxdwqe0LrmoCFhAPNG9Y0ScdjhjOq4ytpCrpQPS4gnTwJ50JnSNH1kL3kuqp0kSWo&__tn__=*NK-R
a_no:	  30 
url:  https://www.facebook.com/hashtag/equity?__eep__=6&__cft__[0]=AZUGbvu0df-ygzeWzg-2AOcDtnOaWXQKTDMHhSfbI5ALwRCh2AAEQWQQCc6NaQXoI_P8rzO6Q6Jd9m7p_MckumNKw5MCyzm9WDHX_u-d_0T9l7VzsF0Bn885vFM95sSqkjsxdwqe0LrmoCFhAPNG9Y0ScdjhjOq4ytpCrpQPS4gnTwJ50JnSNH1kL3kuqp0kSWo&__tn__=*NK-R
a_no:	  31 
url:  https://www.facebook.com/hashtag/stockexchange?__eep__=6&__cft__[0]=AZUGbvu0df-ygzeWzg-2AOcDtnOaWXQKTDMH

a_no:	  0 
url:  https://www.facebook.com/groups/506865050937818/?hoisted_section_header_type=recently_seen&multi_permalinks=899911591633160&__cft__[0]=AZVw7IyZwYwKUE3-ZzawPu8saWVbntEXYUWxdxNbutJZK4wMcuYS2ruSO9Lxya10LO3L917v1vjanfJ7C1DThMZpfW9oV4UpSbBPqiXChevzDkmNJ5840aNfZ0d4z6BkIBXeUt1kQdD1u00z-YOB_2DhYp9_gaEmRKkBJQOFc0l9mFguMg8syxN-aB3RsRgYP1Y&__tn__=%3C%3C%2CP-R
a_no:	  1 
url:  https://www.facebook.com/groups/506865050937818/user/100070018141058/?__cft__[0]=AZVw7IyZwYwKUE3-ZzawPu8saWVbntEXYUWxdxNbutJZK4wMcuYS2ruSO9Lxya10LO3L917v1vjanfJ7C1DThMZpfW9oV4UpSbBPqiXChevzDkmNJ5840aNfZ0d4z6BkIBXeUt1kQdD1u00z-YOB_2DhYp9_gaEmRKkBJQOFc0l9mFguMg8syxN-aB3RsRgYP1Y&__tn__=%3C%2CP-R
a_no:	  2 
url:  https://www.facebook.com/groups/506865050937818/?hoisted_section_header_type=recently_seen&multi_permalinks=899911591633160&__cft__[0]=AZVw7IyZwYwKUE3-ZzawPu8saWVbntEXYUWxdxNbutJZK4wMcuYS2ruSO9Lxya10LO3L917v1vjanfJ7C1DThMZpfW9oV4UpSbBPqiXChevzDkmNJ5840aNfZ0d4z6BkIBXeUt1kQdD1u00z-YOB_2DhYp9_gaEmRKkBJQOFc

new link loaded
has_comments session
sucses1
number of buttons detected:  1
number of buttons detected:  1
has 2 comments
'post_27:',
Post_type:		'general',
is_text_only:		0,
UserName:		'Pakistan Stock Exchange Limited',
Post_time:		'',
No_likes:		'92',
number_comments:	'8',
num_shares:		'5'  ,
Post_content:	'Start your investment journey and trade stocks in Pakistan Stock Market from overseas with Roshan Equity Investment Account. Watch the complete video to learn how to get started today!  ',
Comments:	' comment_1, Naveed Hassan Hingoro : Pakistani citizens can apply in this process |  comment_2, Mohammed Asif Yaseen : Assalam Alikum,Dear investors 45 calls will be shared today so if u want to get these calls and take profit just simply join my paid whatsapp group PSX is gearing up for a massive boomerTo double your income just you have to join my group by paying … See more | ',
https://www.facebook.com/Pakistanstockexchangelimited/videos/712564746702501/?__cft__[0]=AZXP8sCdxzAIJ0u3Z

a_no:	  9 
url:  https://www.facebook.com/hashtag/nifty?__eep__=6&__cft__[0]=AZVwSHNDtoSzRK_x0KyQp8jc2qzRbDUG5aTvJTMZd8Syx5_OfbmOLr6TBFbssgAxt7IBnZZsG3P2BX5x4pBinYuUR44l3OyhYM9EZnj6BuZ37vl3lcTkWUp7YUhyJn1QJF3AlSyAOvPEnc1m6iwGXVdvi0LPApCQgQUltyFJ55uM_FVzB01CpJSLR1W-rTsQL40&__tn__=*NK-R
a_no:	  10 
url:  https://www.facebook.com/hashtag/niftyfifty?__eep__=6&__cft__[0]=AZVwSHNDtoSzRK_x0KyQp8jc2qzRbDUG5aTvJTMZd8Syx5_OfbmOLr6TBFbssgAxt7IBnZZsG3P2BX5x4pBinYuUR44l3OyhYM9EZnj6BuZ37vl3lcTkWUp7YUhyJn1QJF3AlSyAOvPEnc1m6iwGXVdvi0LPApCQgQUltyFJ55uM_FVzB01CpJSLR1W-rTsQL40&__tn__=*NK-R
a_no:	  11 
url:  https://www.facebook.com/hashtag/sharemarket?__eep__=6&__cft__[0]=AZVwSHNDtoSzRK_x0KyQp8jc2qzRbDUG5aTvJTMZd8Syx5_OfbmOLr6TBFbssgAxt7IBnZZsG3P2BX5x4pBinYuUR44l3OyhYM9EZnj6BuZ37vl3lcTkWUp7YUhyJn1QJF3AlSyAOvPEnc1m6iwGXVdvi0LPApCQgQUltyFJ55uM_FVzB01CpJSLR1W-rTsQL40&__tn__=*NK-R
a_no:	  12 
url:  https://www.facebook.com/hashtag/stockmarket?__eep__=6&__cft__[0]=AZVwSHNDtoSzRK_x0KyQp8jc2qzRbDUG5aTvJTMZd8Syx

new link loaded
it had error 1
'post_31:',
Post_type:		'group_post',
is_text_only:		0,
UserName:		'Stock Market Update',
Post_time:		'',
No_likes:		'11',
number_comments:	'',
num_shares:		''  ,
Post_content:	'     ',
Comments:	'',
https://www.facebook.com/photo/?fbid=114957894686791&set=gm.656364609321194&idorvanity=506865050937818&__cft__[0]=AZU2DFnPKvQm7yDTrZcKwBee4mhIgM0E6JGo5NJaefvDSg0cs0gL8aA9VrJ2K96yXFr5S7YlX7Lq9CXhutTy3IFwOhvzFJFnWQfYXT3wdTaauB7NDDJxJlhwrYcJr7gDOVaij8UB3Tz3FnsOXj05omWG7Nrnp3WcMqSrOJRQWtxFfeTVvMjdHa49tUE92cwEDb4&__tn__=EH-R


----------------------------xxxx----------------------------
Searching for stock market Broker
11
has 25  post


----------xx----------

looking inside post no: 0 
<div class="x9f619 x1n2onr6 x1ja2u2z x78zum5 xdt5ytf x193iq5w xeuugli x1r8uery x1iyjqo2 xs83m0k"><h2 class="x1heor9g x1qlqyl8 x1pd3egz x1a2a7pz x193iq5w xeuugli"><span class="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lkfr7t x1lbecb7 x1s688f xzsf02u"><span class="x1lliihq x6ikm8